# Path Planning Project
## Problem Description

In this project we are focusing on self driving car trajectory generation in the Udacity's self driving car simulator. Self driving car is driving on three lane highway in the simulator. The goal is to drive the car as smooth as possible while maintaining the maximal speed of 50 miles per hour. To do that without any accidents it is necessary to pass other cars and adopt speed when necessary. Other important concern is to change lanes or speed as smooth as possible, so that passengers do not feel unsafe or uncomfortable.

## Model Description

C++ model solves the described problem of path generation. The model receives input data from the Udacity's simulator with car coordinates, speed, direction and other cars. Based on this input and map information the model generates a path of 50 coordinates that it sends back to the simulator. Those coordinates represent path that the car will follow until next loop with new set of coordinates.

## Map Information

The CSV file highway_map.csv contains all the map information necessary. This information contains cartesian and freenet coordinates of the road. We can calculate coordinates of a specific lane by calcuating freenet d, for example 2+4*1 is freenet d for the middle lane.

## Data Parsing

The simulator sends JSON object containing car coordinates in freenet and cartesian system, car direction, speed, previous path that is not processed yet and information from other cars on the road (sensor fusion). Those coordinates are then being parsed in appropriate varaibles and arrays.

It is necessary to loop through all cars in sensor_fusion array to determine which cars are in our lane and take appropriate actions, slow down or switch lane if necessary.

We will add some cars to cars_in_front and cars_in_range arrays as well, this arrays will later be used to determine optimal lane.

## Speed Control

Based on the too_close and way_too_close booleans set during sensor_control array loop we control the desired speed (ref_vel variable).

Reference velocity - ref_vel is increased or decreased incrementally to avoid high jerk during acceleration or breaking.

## Search of Optimal Lane

If there is a slower vehicle in front of us it is desireable to switch to faster lane as well. This is why we set switch_lane boolean in case of slower car in front of us during sensor_fusion array loop.

The model checks every 30 simulator iterations if the switch_lane boolean is active and searches optimal lane in case that it is.

### PTG.Lane cost function

PTG.Lane cost function returns optimal lane and it is part of PTG class (PTG.cpp, PTG.h).

The function first separates previosly collected cars_in_front and cars_in_range vectors. Each of the vecotrs is separated into three spearate vectors depending on the lane in which cars are in. As result we get 2 arrays: 
cars_in_lane_front[3] - Array used to determine fastest lane.
cars_in_lane_range[3] - Array used to determine collision risk.

After previosuly described arrays have been extracted, we can start calculating cost of each lane to determine lane with lowest cost. 

This is done by looping through each lane and adding costs calculated by three cost functions described later.

It is important to note that if we need to switch 2 lanes, for example from left lane to right lane, then we need to consider collision risk in the middle lane as well.

### slower_car_in_lane_cost Function

slower_car_in_lane_cost function runs for each lane to add higher cost to lanes with closer vehicles. Function predicts where vehicle from sensor_fusion array will be 0.2 seconds in future.

If a vehicle is closer to the car more cost will be added. If vehicle does not exist, then cost will be 0.

### collision_cost Function

collision_cost function makes sure that we do not collide with some other vehicle during lane switch. If there is vehicle in lane where we plan to switch and there is collision risk with that vehicle (car_s is in range of check_car_s), very high cost will be added.

### lane_switching_cost Fuction

lane_switching_cost function adds small cost for lane switch to avoid that car constantly switches lanes when costs of two lanes are basically the same.

### Returning Optimal Lane

At the end optimal lane with lowest cost will be returned. If we need to switch 2 lanes, for example from left lane to right lane, then the middle lane will be returned first. We do this to avoid high jerk due to switching two lanes at same time and because it is not usual and expected from other drivers that a car switches two lanes at once.

## Path Generation

After optimal lane has been choosen, it is necessary to generate path on which the car will drive. Final path will be sent as an array of x and y coordiates to the simulator.


### Get Reference Cordinates

First we need to get starting coordinates that we will use as reference points. We will add them to ptsx and ptsy vectors. If there are no previous coordinates returned from the simulator available, we will use car's current coordinates as a starting point. If there are at least two cordinates in previous_path vectors available, we will use them as a starting point and to calculate yaw rate.


### Create List of Evenly Spaced Corrdinates

To simplify the math we will calculate waypoints in freenet coordinate system, by adding 30 meters to the "s" axis. "d" axis is caluclated by multiplying current (optimal) lane with 4 and adding 2. For example "d" axis for the lane 0 (left) is 2+4*0 = 2. 

With help of the getXY function we get carthesian coordinates that can later be used by the simulator.  

The last step before calculating the detailed path with spline is rotating carthesian coordinates that we have just calculated to car reference angle of 0 degrees. We do this to simlify the math.

### Spline function



### Returning Data to the Simulator

## Conclusion